<a href="https://colab.research.google.com/github/gemma24/ISYS5002-assessment/blob/main/Sandbox_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Client's requirements
This notebook able to:
* Prompts for a company's stock market ticker symbol
* Scrapes the Yahoo finance website to extract info on the key company
* Calculate the each executive's expected income tax based on the ATO tax table
* Data can be saved as a CSV or SQLite database
* Follow PEP8 conventions



# Problem Solving Methodology
In the process of problem-solving, there are five key steps to follow:
1. State the problem clearly
2. Describe the input and output information
3. Work the problem by hand 
4. Develop an algorithm
5. Test solution with a variety of data.